<a href="https://colab.research.google.com/github/andersknudby/Teledetection/blob/master/Chapitre_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Visualisation des données
Une chose vraiment utile à faire en Python est de pouvoir visualiser vos résultats. Ainsi, vous n'avez pas toujours besoin d'exporter des données vers des fichiers et de les ouvrir dans QGIS pour évaluer ce que vous avez fait. Dans ce chapitre, nous verrons comment visualiser des données matricielles et vectorielles, et comment créer des graphiques simples avec ces données.

##Démarrage
Pour commencer, nous allons récupérer l'image, les points et les polygones avec lesquels nous avons travaillé dans les chapitres précédents. Rien de nouveau ici, il s'agit juste de préparer les données pour travailler, à l'exception d'une chose : nous importons la fonction **pyplot** de la bibliothèque **matplotlib**.

In [ ]:
!pip install rasterio
!pip install geopandas
!pip install rasterstats

import rasterio
import geopandas as gpd
import rasterstats

import numpy as np
from shapely.geometry import Point
from matplotlib import pyplot as plt


from google.colab import drive
drive.mount('/content/drive')

myDir = '/content/drive/My Drive/Python files/'

import os
if os.path.exists(myDir + 'sfu.tif'):
  print("Drive mounted and directory found")
else:
  print("No access to the files")

##Traçage de données matricielles
Maintenant, lisez l'image. Rappelez-vous que la **bande 1 est la "bande rouge**", puis le vert, puis le bleu :

In [ ]:
imageFilename = myDir + 'sfu.tif'
ds = rasterio.open(imageFilename)  # ds is a commonly used shorthand for 'dataset'

red = ds.read(1).astype('uint16')
green = ds.read(2).astype('uint16')
blue = ds.read(3).astype('uint16')

Nous pouvons utiliser la fonction pyplot pour afficher facilement une seule bande :

In [ ]:
pyplot.imshow(red)


Les couleurs sont un peu bizarres dans l'image ci-dessus, qui utilise la carte de couleurs par défaut de matplotlib. Elles ne sont pas mauvaises, mais rappelez-vous que nous avons tracé la bande rouge ici, donc les pixels **clairs** sont en fait ceux qui reflètent beaucoup de lumière rouge. Lorsque l'on trace une seule bande, je préfère normalement utiliser une échelle de gris à la place, pour éviter toute indication que l'on regarde des données multispectrales :

In [ ]:
pyplot.imshow(red, cmap="Greys_r")

Cependant, il s'agit en fait d'une image à 3 bandes, donc pour une interprétation visuelle, nous pourrions tout aussi bien tracer les trois bandes dans un composite en couleurs réelles. À cette fin, rasterio dispose d'une fonction spéciale que nous allons importer et utiliser pour produire une image qui a l'air plutôt jolie :

In [ ]:
from rasterio.plot import show
show(ds)

##Plotting points data
Passons au traçage des fichiers vectoriels. Nous allons commencer par lire les données **points** :

In [ ]:
pointsFilename = myDir + "points.shp"
pts = gpd.read_file(pointsFilename)
pts

Et de les tracer :

In [ ]:
fig, ax = plt.subplots()
pts.plot(ax=ax)
plt.title("Points")
plt.show()

Il existe de nombreuses options de traçage, par exemple pour modifier la couleur, la taille et la transparence des points :

In [ ]:
fig, ax = plt.subplots()
pts.plot(ax=ax, color="green", markersize=100, alpha=0.2)
plt.title("Points")
plt.show()

##Tracer des données de polygones
Cela fonctionne de la même manière pour les polygones :

In [ ]:
polygonsFilename = myDir + "polygons.shp"
polys = gpd.read_file(polygonsFilename)  # I often use 'pts' as short for 'points' This is fairly common.
fig, ax = plt.subplots()
polys.plot(ax=ax)
plt.title("Polygons")
plt.show()

Ou, pour tracer les polygones sans remplissage, ce qui est souvent utile si vous voulez voir ce qu'il y a à l'intérieur (cela sera utile plus tard) :

In [ ]:
fig, ax = plt.subplots()
polys.plot(ax=ax, facecolor="none", edgecolor='blue', lw=0.7)
plt.title("Polygons")
plt.show()

Il s'agit maintenant d'assembler le tout, afin de voir où se trouvent les points et les polygones par rapport à l'image, et les uns par rapport aux autres :

In [ ]:
fig, ax = plt.subplots(1, figsize=(15,15))  # The figsize argument makes the plot a bit bigger, so we can see better
show(ds, ax=ax)  # This puts the image in the plot
pts.plot(ax=ax, color="red")  # ... and the points
polys.plot(ax=ax, facecolor="none", edgecolor='blue', lw=1.2)  # ... and the polygons

##Travailler avec des transects
Les transects, des lignes qui traversent des gradients, peuvent être utiles pour illustrer des changements graduels ou abrupts dans l'environnement. Ici, nous allons utiliser un transect pour montrer comment les valeurs CCV changent le long de la route qui longe le bord est de l'image.

Tout d'abord, nous créons le transect et le traçons pour nous assurer que nous le plaçons au bon endroit :

In [ ]:
from shapely.geometry import LineString

transect = LineString([(507050, 5458400), (507100, 5458400)])  # Create the geometry
gdf = gpd.GeoDataFrame(geometry=[transect])  # Turn it into a geodataframe for plotting

# Then plot it with the image
fig, ax = plt.subplots(1, figsize=(15,15))
show(ds, ax=ax)
gdf.plot(ax=ax, color="red")

Ok, maintenant nous savons que le transect est au bon endroit. Mais il n'y a pas de bonne fonction pour extraire les valeurs de tous les pixels sous une ligne, donc ce que nous allons faire à la place est de **séparer la ligne en points**, et ensuite utiliser la fonction **sample** de **rasterio** (qui est basée sur des points), comme nous l'avons fait dans le chapitre précédent :

In [ ]:
from shapely.geometry import MultiPoint
from shapely.ops import split

# The line is 50m long, and the pixels in the image are 10cm across
# So if we want to sample every pixel we need to turn the line into 501 points (point at both ends)
nPoints = 501

# We can use the 'interpolate' function to create a set of evenly spaced points along out transect
multiPoint = MultiPoint([transect.interpolate((i/nPoints), normalized=True) for i in range(nPoints)])

# Before we can use it with the sample function, we need to turn this MultiPoint object into a list of coordinate pairs
pointsAsList = [(p.x, p.y) for p in multiPoint]

# And now we can sample the values from the imagery, in the red, green, and blue bands
redValues = [x[0] for x in ds.sample(pointsAsList, indexes=1)]
greenValues = [x[0] for x in ds.sample(pointsAsList, indexes=2)]
blueValues = [x[0] for x in ds.sample(pointsAsList, indexes=3)]

# To take a quick look and check that there is indeed 501 numbers here:
print(*redValues)

Ok, maintenant il ne reste plus que la visualization :

In [ ]:
# To make a nice plot, we need to isolate the x values
xValues = [p.x for p in multiPoint]

# And we'll put it all in a dataframe. This isn't really necessary, but makes plotting easier
data = {'xValues': xValues, 'redValues': redValues, 'greenValues': greenValues, 'blueValues': blueValues} 
df = pd.DataFrame(data) 

fig, ax = plt.subplots(1, figsize=(11,11))
ax.plot(xValues, redValues, color="red")
ax.plot(xValues, greenValues, color="green")
ax.plot(xValues, blueValues, color="blue")

##Exercice
Dans un nouveau cahier, modifiez le code de ce chapitre pour tracer comment le CCV change à travers le transect.